# Setting up dataset

In [ ]:
import pandas as pd

schools_complete_df = pd.read_csv("schools_complete.txt")
students_complete_df = pd.read_csv("students_complete.txt")
merged_df = schools_complete_df.merge(students_complete_df, how="left", on="school_name")

school_merge = merged_df.groupby("school_name")
school_avg_reading_score = school_merge["reading_score"].mean()
school_avg_math_score = school_merge["math_score"].mean()
school_total_score = school_merge["reading_score"].count()

schools_complete_df.set_index("school_name", inplace=True)

combined_df = pd.DataFrame({"type":schools_complete_df["type"], "budget":schools_complete_df["budget"], 
                            "reading_score":school_avg_reading_score, "math_score":school_avg_math_score,})
combined_df

,type,budget,reading_score,math_score
school_name,,,,
Bailey High School,District,3124928,81.033963,77.048432
Cabrera High School,Charter,1081356,83.975780,83.061895
Figueroa High School,District,1884411,81.158020,76.711767
Ford High School,District,1763916,80.746258,77.102592
Griffin High School,Charter,917500,83.816757,83.351499
Hernandez High School,District,3022020,80.934412,77.289752
Holden High School,Charter,248087,83.814988,83.803279
Huang High School,District,1910635,81.182722,76.629414
Johnson High School,District,3094650,80.966394,77.072464


In [ ]:
combined_df["budget"].max(), combined_df["budget"].min()

(3124928, 248087)

# Actually Binning!

What we want to do is compare schools that have similar budgets. However, right now, you can see that the budgets are all unique numbers, so we can't group them.

One way of doing this is to group schools into categories--or "binning" them. 

In [ ]:
bins = [0, 250000,  500000,  700000, 1000000, 2000000, 3000000, 4000000]

In [ ]:
pd.cut(combined_df["budget"], bins)

school_name
Bailey High School       (3000000, 4000000]
Cabrera High School      (1000000, 2000000]
Figueroa High School     (1000000, 2000000]
Ford High School         (1000000, 2000000]
Griffin High School       (700000, 1000000]
Hernandez High School    (3000000, 4000000]
Holden High School              (0, 250000]
Huang High School        (1000000, 2000000]
Johnson High School      (3000000, 4000000]
Pena High School           (500000, 700000]
Rodriguez High School    (2000000, 3000000]
Shelton High School      (1000000, 2000000]
Thomas High School       (1000000, 2000000]
Wilson High School       (1000000, 2000000]
Wright High School       (1000000, 2000000]
Name: budget, dtype: category
Categories (7, interval[int64, right]): [(0, 250000] < (250000, 500000] < (500000, 700000] <
                                         (700000, 1000000] < (1000000, 2000000] < (2000000, 3000000] <
                                         (3000000, 4000000]]

Basically, how this works is we create a list of bins:

`[0, 250000,  500000,  700000, 1000000, 2000000, 3000000, 4000000]`

We give these to pandas' cut function, along with the column we want to group into categories ( `combined_df["budget"]` ).

You'll notice that what pd.cut does is go through the rows and it tries to figure out which numbers it goes between. 

For example, Bailey Highschool is `(3000000, 4000000]` because the original value is `3124928`--between `3000000` and `4000000`.

This isn't really efficient, of course, so we can add labels to the mix.

In [ ]:
bins = [0, 250000,  500000,  700000, 1000000, 2000000, 3000000, 4000000]
labels = ["<250,000", "250,001 - 500,000", "500,001 - 700,000", "700,001 - 1,000,000", "1,000,000 - 2,000,000", "2,000,000 - 3,000,000", "3,000,000 - 4,000,000"]

pd.cut(combined_df["budget"], bins, labels=labels)

school_name
Bailey High School       3,000,000 - 4,000,000
Cabrera High School      1,000,000 - 2,000,000
Figueroa High School     1,000,000 - 2,000,000
Ford High School         1,000,000 - 2,000,000
Griffin High School        700,001 - 1,000,000
Hernandez High School    3,000,000 - 4,000,000
Holden High School                    <250,000
Huang High School        1,000,000 - 2,000,000
Johnson High School      3,000,000 - 4,000,000
Pena High School             500,001 - 700,000
Rodriguez High School    2,000,000 - 3,000,000
Shelton High School      1,000,000 - 2,000,000
Thomas High School       1,000,000 - 2,000,000
Wilson High School       1,000,000 - 2,000,000
Wright High School       1,000,000 - 2,000,000
Name: budget, dtype: category
Categories (7, object): ['<250,000' < '250,001 - 500,000' < '500,001 - 700,000' <
                         '700,001 - 1,000,000' < '1,000,000 - 2,000,000' < '2,000,000 - 3,000,000' <
                         '3,000,000 - 4,000,000']

Notice that the labels will always have one less than bins. The reason why is that bins starts at 0 and the first label is numbers between 0 and 250,000.

Since these are strings, we can use them to do a groupby.

In [ ]:
combined_df["budget_bins"] = pd.cut(combined_df["budget"], bins, labels=labels)
combined_df

,type,budget,reading_score,math_score,budget_bins
school_name,,,,,
Bailey High School,District,3124928,81.033963,77.048432,"3,000,000 - 4,000,000"
Cabrera High School,Charter,1081356,83.975780,83.061895,"1,000,000 - 2,000,000"
Figueroa High School,District,1884411,81.158020,76.711767,"1,000,000 - 2,000,000"
Ford High School,District,1763916,80.746258,77.102592,"1,000,000 - 2,000,000"
Griffin High School,Charter,917500,83.816757,83.351499,"700,001 - 1,000,000"
Hernandez High School,District,3022020,80.934412,77.289752,"3,000,000 - 4,000,000"
Holden High School,Charter,248087,83.814988,83.803279,"<250,000"
Huang High School,District,1910635,81.182722,76.629414,"1,000,000 - 2,000,000"
Johnson High School,District,3094650,80.966394,77.072464,"3,000,000 - 4,000,000"


Here, for example, we're looking at the reading scores per amount spent!

In [ ]:
combined_df.groupby("budget_bins")["reading_score"].mean()

budget_bins
<250,000                 83.814988
250,001 - 500,000              NaN
500,001 - 700,000        84.044699
700,001 - 1,000,000      83.816757
1,000,000 - 2,000,000    82.822740
2,000,000 - 3,000,000    80.744686
3,000,000 - 4,000,000    80.978256
Name: reading_score, dtype: float64